#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader=WebBaseLoader("https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier")
loader

In [6]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier', 'title': 'Optimize a classifier | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'This tutorial walks through optimizing a classifier based on user a feedback.', 'language': 'en'}, page_content='\n\n\n\n\nOptimize a classifier | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringTutorialsOptimize a classifierHow-to GuidesPlaygroundPromptsConceptual GuideDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referencePrompt EngineeringTutorialsOptimize a classifierOn this pageOptimize a classifier\nThis tutorial 

In [7]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [8]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier', 'title': 'Optimize a classifier | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'This tutorial walks through optimizing a classifier based on user a feedback.', 'language': 'en'}, page_content='Optimize a classifier | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier', 'title': 'Optimize a classifier | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'This tutorial walks through optimizing a classifier based on user a feedback.', 'language': 'en'}, page_content='Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringTutorialsOptimize a classifierHow-to GuidesPlaygroundPromptsConceptual GuideDeployment (Lan

In [10]:
len(documents)

14

In [11]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [12]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [13]:
vectorstoredb

In [16]:
## Query From a vector db
query="send trace to specific project"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'That is exactly what we will do in this example.\nThe objective‚Äã\nIn this example, we will build a bot that classify GitHub issues based on their title.\nIt will take in a title and classify it into one of many different classes.\nThen, we will start to collect user feedback and use that to shape how this classifier performs.\nGetting started‚Äã\nTo get started, we will first set it up so that we send all traces to a specific project.\nWe can do this by setting an environment variable:\nimport osos.environ["LANGCHAIN_PROJECT"] = "classifier"\nWe can then create our initial application. This will be a really simple function that just takes in a GitHub issue title and tries to label it.'

In [18]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [20]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x13a364a10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x13a3667e0>, root_client=<openai.OpenAI object at 0x13a33b140>, root_async_client=<openai.AsyncOpenAI object at 0x13a364a70>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_fact

In [21]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"send trace to specific project",
    "context":[Document(page_content="To get started, we will first set it up so that we send all traces to a specific project. We can do this by setting an environment variable:")]
})

'The context provided instructs on setting up the process to send all traces to a specific project by setting an environment variable. However, it does not contain a specific question to answer. Could you please provide the question you would like answered based on this context?'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [22]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [23]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [24]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x139db7c50>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatOpe

In [30]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"collect feedback in two forms"})
response['answer']

'How can positive feedback be used in future iterations according to the context?'

In [31]:

response

{'input': 'collect feedback in two forms',
 'context': [Document(id='1d6e045c-8181-4d24-b33b-c9aed35ec464', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier', 'title': 'Optimize a classifier | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'This tutorial walks through optimizing a classifier based on user a feedback.', 'language': 'en'}, page_content='We can then start to interact with it.\nWhen interacting with it, we will generate the LangSmith run id ahead of time and pass that into this function.\nWe do this so we can attach feedback later on.\nHere\'s how we can invoke the application:\nrun_id = uuid.uuid4()topic_classifier(    "fix bug in LCEL",    langsmith_extra={"run_id": run_id})\nHere\'s how we can attach feedback after.\nWe can collect feedback in two forms.\nFirst, we can collect "positive" feedback - this is for examples that the model got right.\nls_client = Client()run_id = uuid.uuid4()topic_classifier(    "fix 

In [32]:
response['context']

[Document(id='1d6e045c-8181-4d24-b33b-c9aed35ec464', metadata={'source': 'https://docs.smith.langchain.com/prompt_engineering/tutorials/optimize_classifier', 'title': 'Optimize a classifier | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'This tutorial walks through optimizing a classifier based on user a feedback.', 'language': 'en'}, page_content='We can then start to interact with it.\nWhen interacting with it, we will generate the LangSmith run id ahead of time and pass that into this function.\nWe do this so we can attach feedback later on.\nHere\'s how we can invoke the application:\nrun_id = uuid.uuid4()topic_classifier(    "fix bug in LCEL",    langsmith_extra={"run_id": run_id})\nHere\'s how we can attach feedback after.\nWe can collect feedback in two forms.\nFirst, we can collect "positive" feedback - this is for examples that the model got right.\nls_client = Client()run_id = uuid.uuid4()topic_classifier(    "fix bug in LCEL",    langsmith_extra={"run_id": run_id})ls_